In [1]:
import numpy as np
import os
import sys
import collections
import matplotlib.pyplot as plt
import gzip
import loompy
import scipy.sparse as sparse
import urllib.request
import pybedtools
import warnings
from sklearn.neighbors import NearestNeighbors
from matplotlib.collections import LineCollection

import cytograph as cg
from cytograph.decomposition import HPF
from scipy.stats import poisson
from cytograph.manifold import BalancedKNN
from cytograph.metrics import jensen_shannon_distance
from cytograph.embedding import tsne
from cytograph.clustering import PolishedLouvain, PolishedSurprise
from cytograph.plotting import manifold

sys.path.append('/home/camiel/chromograph')
from chromograph.plotting.QC_plot import QC_plot

from umap import UMAP
import sklearn.metrics
from scipy.spatial import distance
import community
import networkx as nx
from scipy import sparse
from typing import *

import timeit
import time
import numba
from numba import njit, prange

from sklearn.decomposition import IncrementalPCA
from pynndescent import NNDescent

In [2]:
## Parameters
f = '/data/proj/scATAC/chromograph/build_20191206/Cerebellum.loom'
outdir = '/data/proj/scATAC/chromograph/tmp'

In [3]:
import logging

logger = logging.getLogger()
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%H:%M:%S')

In [4]:
ds = loompy.connect(f, mode='r+')
print(ds.shape)
blayer = '5kb_bins'

(617643, 17811)


In [5]:
if not os.path.isdir(outdir):
    os.mkdir(outdir)

In [6]:
bins = (ds.ra['Coverage'] > -2) & (ds.ra['Coverage'] < 2)

In [9]:
data = ds[blayer][:,:]
data = data[bins,:1000]
data.shape

(560008, 1000)

In [10]:
np.savetxt(os.path.join(outdir, "Cerebellum.csv"), data, delimiter=",")

In [ ]:
# from numba import njit, prange
# @njit(parallel=True)
# def numba_parallel_jaccard(M):
#     #here if first it is run with range, then it is very slow.
#     #then one range is replaced with prange, then it crashes, and then to range again
#     #and then it is fastest (3x faster than with prange)
#     u, i = M.shape
#     js = np.zeros((u,u))
#     for i in prange(u):
#         for j in range(u):
#             v = (M[i]*M[j] > 0).sum()
#             if v:
#                 js[i,j] = v/((M[i] + M[j] > 0).sum())
#             else: 
#                 js[i,j] = 0
#     return js

# t = []
# for x in range(5):
#     start = time.time()

#     N = 100
#     srt = ds.ca['Clusters'][:N].argsort()

#     ini = numba_parallel_jaccard(np.array([[0,1], [1,0]]))
#     js = numba_parallel_jaccard(data[:,:N].T)

#     t.append(time.time() - start)
#     print(time.time() - start)
    
# plt.imshow(js[srt,:][:,srt])    
# print(f'mean {np.mean(t)} with st. dev: {np.std(t)}')

In [36]:
# @numba.jit("float32(float64[:], float64[:])", nopython=True, cache=True)
# @njit(parallel=True, cache=True)

@njit(parallel=True, cache=True)
def jaccard_sim(data) -> float:
    
    N = data.shape[1]
    f = data.shape[0]
    js = np.zeros((N,N))

    for i in range(N):
        v1 = data[:,i]
        for j in range(i+1, N):
            v2 = data[:,j]
    
            p = np.zeros(f)
            qr = np.zeros(f)
            for x in range(f):
                if v1[x] > 0 and v2[x] > 0:
                    p[x] = 1
                elif v1[x] > 0 or v2[x] > 0:
                    qr[x] = 1

            sp = np.sum(p) 
            sqr = np.sum(qr)
            js[i,j] = sp / (sp+sqr)
            js[j,i] = js[i,j]
            
    return js

# t = []
# for x in range(1):
#     start = time.time()

#     N = 10
#     srt = ds.ca['Clusters'][:N].argsort()

#     js = jaccard_sim(data[:,:N])


#     t.append(time.time() - start)
#     print(time.time() - start)

# plt.imshow(js[srt,:][:,srt])    
# print(f'mean {np.mean(t)} with st. dev: {np.std(t)}')
    
N = 100

srt = ds.ca['Clusters'][:N].argsort()
js = jaccard_sim(data[:,:N])
plt.imshow(js[srt,:][:,srt])

KeyboardInterrupt: 

In [ ]:
C = np.sum(data, axis=0)
P = C / data.shape[0]

E = np.zeros([N,N])

for i in range(N):
    Pi = P[i]
    for j in range(i+1, N):
        Pj = P[j]
        E[i,j] = 1/(Pi**-1 + Pj**-1 -1)
        E[j,i] = E[i,j]

M = js - E
        
fig = plt.figure(figsize=(8,24))
ax1 = fig.add_subplot(131)
ax1.imshow(js[srt,:][:,srt])
ax1.set_title('Jaccard Similarity')

ax2 = fig.add_subplot(132)
ax2.imshow(E[srt,:][:,srt])
ax2.set_title('Expected Similarity')

ax3 = fig.add_subplot(133)
ax3.imshow(M[srt,:][:,srt])
ax3.set_title('Residual Similarity')

In [ ]:
## pynndescent

index = NNDescent(data.T, metric='jaccard')

In [37]:
?NNDescent

Init signature:
NNDescent(
    data,
    metric='euclidean',
    metric_kwds=None,
    n_neighbors=15,
    n_trees=None,
    leaf_size=None,
    pruning_level=0,
    tree_init=True,
    random_state=<module 'numpy.random' from '/home/camiel/anaconda3/envs/chromograph/lib/python3.7/site-packages/numpy/random/__init__.py'>,
    algorithm='standard',
    low_memory=False,
    max_candidates=20,
    n_iters=None,
    delta=0.001,
    rho=0.5,
    n_jobs=None,
    seed_per_row=False,
    verbose=False,
)
Docstring:     
NNDescent for fast approximate nearest neighbor queries. NNDescent is
very flexible and supports a wide variety of distances, including
non-metric distances. NNDescent also scales well against high dimensional
data in many cases. This implementation provides a straightfoward
interface, with access to some tuning parameters.

Parameters
----------
data: array os shape (n_samples, n_features)
    The training data set to find nearest neighbors in.

metric: string or callable (

In [ ]:
## Sparse jaccard

In [18]:
spm = ds.layers['5kb_bins'].sparse(rows = bins)

rows = spm.row
cols = spm.col

print(rows)

[     0      0      0 ... 566822 566822 566822]


In [27]:
v1 = rows[cols==0]
v2 = rows[cols==1]

pqr = len(np.unique(np.concatenate((v1,v2))))
p = (len(np.unique(v1)) + len(np.unique(v2))) - pqr
print(p, pqr)
print(p/pqr)

12606 79466
0.1586338811567211
12606 79466
0.1586338811567211
12606 79466
0.1586338811567211
12606 79466
0.1586338811567211
12606 79466
0.1586338811567211
12606 79466
0.1586338811567211
12606 79466
0.1586338811567211
12606 79466
0.1586338811567211
512 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
@njit()
def jac_sim_sparse(v1: np.ndarray, v2: np.ndarray) -> float:
    ## Total features
    pqr = len(np.unique(np.concatenate((v1,v2))))
    
    ## overlap
    p = (len(np.unique(v1)) + len(np.unique(v2))) - pqr
    
    return p/pqr

t = []
for x in range(5):
    start = time.time()

    N = 100
    srt = ds.ca['Clusters'][:N].argsort()

    js = np.zeros([N,N])

    for i in range(N):
        v1 = rows[cols==i]
        for j in range(i+1, N):
            v2 = rows[cols==j]
            js[i,j] = jac_sim_sparse(v1,v2)
            js[j,i] =  js[i,j]

#     plt.imshow(js[srt,:][:,srt])

    t.append(time.time() - start)
    print(time.time() - start)
plt.imshow(js[srt,:][:,srt])    
print(f'mean {np.mean(t)} with st. dev: {np.std(t)}')
print(js[0,1])

0
1
2
3
4
5


KeyboardInterrupt: 

In [25]:
def jac_sim_sparse(v1: np.ndarray, v2: np.ndarray) -> float:
    ## Total features
    pqr = len(set(np.concatenate((v1,v2))))
    
    ## overlap
    p = (len(set(v1)) + len(set(v2))) - pqr

    return p/pqr

t = []
for x in range(5):
    start = time.time()

    N = 100
    srt = ds.ca['Clusters'][:N].argsort()

    js = np.zeros([N,N])

    for i in range(N):
        v1 = rows[cols==i]
        for j in range(i+1, N):
            v2 = rows[cols==j]
            js[i,j] = jac_sim_sparse(v1,v2)
            js[j,i] =  js[i,j]

#     plt.imshow(js[srt,:][:,srt])

    t.append(time.time() - start)
    print(time.time() - start)
plt.imshow(js[srt,:][:,srt])    
print(f'mean {np.mean(t)} with st. dev: {np.std(t)}')
print(js[0,1])

0
1
2
3
4
5


KeyboardInterrupt: 

In [ ]:
print(np.max(js), np.min(js))
print(np.max(E), np.min(E))
print(np.max(M), np.min(M))

p as the number of variables that are positive in both vectors  
q as the number of variables that are positive in the first and negative in the second  
r as the number of variables that are negative in the first and positive in the second  
s as the number of variables that are negative in both  
t as the total number of variables (which is also p+q+r+s, as those are all exclusive)